In [2]:
import pyptvgtfs
import pandas as pd
import os
from shapely.geometry import Point

In [3]:
ptv = pyptvgtfs.process_gtfs_zip('http://data.ptv.vic.gov.au/downloads/gtfs.zip', '')
ptv = ptv.set_index(['branch_id', 'table_name'], inplace=False)['df']
# 30s

In [6]:
branch_ids = ptv.index.get_level_values('branch_id').unique()
table_names = ptv.index.get_level_values('table_name').unique()

In [9]:
os.makedirs('../data/ptv/20240224', exist_ok=True)

In [10]:
for branch_id in branch_ids:
    for table_name in table_names:
        df = ptv[branch_id][table_name]
        
        total_rows = len(df)

        chunk_size = 1000000

        num_chunks = -(-total_rows // chunk_size)  # Ceiling division to ensure all rows are covered

        # Split the DataFrame into chunks and save each chunk to a separate CSV file
        for i, chunk_start in enumerate(range(0, total_rows, chunk_size), 1):
            chunk_end = min(chunk_start + chunk_size, total_rows)
            chunk = df.iloc[chunk_start:chunk_end]
            chunk.to_csv(f'../data/ptv/20240224/ptv-{branch_id}-{table_name}-{i}.csv', index=False)
            print(f'Chunk {branch_id}-{table_name}-{i} saved.')

Chunk 1-agency-1 saved.
Chunk 1-calendar-1 saved.
Chunk 1-calendar_dates-1 saved.
Chunk 1-routes-1 saved.
Chunk 1-shapes-1 saved.
Chunk 1-shapes-2 saved.
Chunk 1-stops-1 saved.
Chunk 1-stop_times-1 saved.
Chunk 1-trips-1 saved.
Chunk 10-agency-1 saved.
Chunk 10-calendar-1 saved.
Chunk 10-calendar_dates-1 saved.
Chunk 10-routes-1 saved.
Chunk 10-shapes-1 saved.
Chunk 10-stops-1 saved.
Chunk 10-stop_times-1 saved.
Chunk 10-trips-1 saved.
Chunk 11-agency-1 saved.
Chunk 11-calendar-1 saved.
Chunk 11-calendar_dates-1 saved.
Chunk 11-routes-1 saved.
Chunk 11-shapes-1 saved.
Chunk 11-stops-1 saved.
Chunk 11-stop_times-1 saved.
Chunk 11-trips-1 saved.
Chunk 2-agency-1 saved.
Chunk 2-calendar-1 saved.
Chunk 2-calendar_dates-1 saved.
Chunk 2-routes-1 saved.
Chunk 2-shapes-1 saved.
Chunk 2-stops-1 saved.
Chunk 2-stop_times-1 saved.
Chunk 2-trips-1 saved.
Chunk 3-agency-1 saved.
Chunk 3-calendar-1 saved.
Chunk 3-calendar_dates-1 saved.
Chunk 3-routes-1 saved.
Chunk 3-shapes-1 saved.
Chunk 3-stops-

In [ ]:
ptv['5']['shapes']

In [ ]:
for branch_id in branch_ids:
    ptv[branch_id]['stops']['branch_id'] = branch_id

In [ ]:
[ptv[branch_id]['stops']['stop_id'].is_unique for branch_id in branch_ids]

In [ ]:
df_stops : pd.DataFrame = pd.concat([ptv[branch_id]['stops'] for branch_id in branch_ids])

In [ ]:
gdf_vic = gdf[gdf['STE_CODE21'] == '2']

In [ ]:
df_stops_points = df_stops.set_index(['stop_id', 'branch_id']).apply(lambda x: Point(x['stop_lon'], x['stop_lat']), axis=1)

In [ ]:
df_stmb : pd.DataFrame = df_stops_points.apply(lambda p: gdf_vic[gdf_vic.geometry.contains(p)]['MB_CODE21'].unique())
# 5m - 10m

In [ ]:
df_stmb.apply(lambda x: len(x)).value_counts()

In [ ]:
df_stmbx : pd.Series = df_stmb.apply(lambda x: x[0] if len(x) == 1 else None)
df_stmbx.rename('mb_id', inplace=True)
df_stmbx : pd.DataFrame = df_stmbx.to_frame().reset_index()

In [ ]:
df_stops['mb_id'] = df_stmbx['mb_id']

In [ ]:
os.makedirs('../data/ptv/', exist_ok=True)

In [ ]:
df_stops.to_csv('../data/ptv/stops_mb.csv', index=False)